In [1]:
import glob
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras 
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import *
import matplotlib.pyplot as plt

In [2]:
smile = glob.glob("positives/*.jpg")
no_smile = glob.glob("negatives/*.jpg")

In [3]:
images=[]
labels=[]

for image in smile:
    image = load_img(image, target_size=(32,32), color_mode='grayscale')
    image = img_to_array(image)
    label = 1 
    label = float(label)
    images.append(image)
    labels.append(label)
    
for image in no_smile:
    image = load_img(image, target_size=(32,32), color_mode='grayscale')
    image = img_to_array(image)
    label = 0
    label = float(label)
    images.append(image)
    labels.append(label)

In [4]:
X = np.array(images)
y = np.array(labels)

In [5]:
X.shape

(13165, 32, 32, 1)

In [6]:
y.shape

(13165,)

In [7]:
X /= 255.0
total = len(y)
total_positive = np.sum(y)
total_negative = total - total_positive

In [8]:
(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=0.2,stratify=y, random_state=999)

In [9]:
(X_train, X_val, y_train, y_val) = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=999)

In [10]:
data_augmentation = keras.Sequential(
 [
 layers.RandomFlip("horizontal"),
 layers.RandomRotation(0.1),
 layers.RandomZoom(0.2),
 ]
)

In [11]:
inputs = Input(shape=(32, 32, 1))
x = data_augmentation(inputs)
x = Conv2D(filters=20,kernel_size=(5, 5), padding='same', strides=(1, 1))(x)
x = ELU()(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
x = Dropout(0.4)(x)
x = Conv2D(filters=50, kernel_size=(5, 5), padding='same', strides=(1, 1))(x)
x = ELU()(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
x = Dropout(0.4)(x)
x = Flatten()(x)
x = Dense(units=500)(x)
x = ELU()(x)
x = Dropout(0.4)(x)
output = Dense(1, activation='sigmoid')(x)
model = Model(inputs=inputs, outputs=output)

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 1)]       0         
                                                                 
 sequential (Sequential)     (None, 32, 32, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 20)        520       
                                                                 
 elu (ELU)                   (None, 32, 32, 20)        0         
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 20)       80        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 20)       0         
 )                                                           

In [13]:
model.compile(loss="binary_crossentropy",
 optimizer="rmsprop",
 metrics=["accuracy"])

In [14]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32,
         class_weight={ 1.0: total / total_positive, 0.0: total / total_negative})

Epoch 1/20
264/264 [==============================] - 15s 46ms/step - loss: 2.0822 - accuracy: 0.6364 - val_loss: 0.5729 - val_accuracy: 0.7499
Epoch 2/20
264/264 [==============================] - 12s 44ms/step - loss: 1.2064 - accuracy: 0.7398 - val_loss: 0.3570 - val_accuracy: 0.8619
Epoch 3/20
264/264 [==============================] - 12s 47ms/step - loss: 1.0656 - accuracy: 0.7671 - val_loss: 0.7251 - val_accuracy: 0.6773
Epoch 4/20
264/264 [==============================] - 13s 48ms/step - loss: 0.9605 - accuracy: 0.7918 - val_loss: 0.6259 - val_accuracy: 0.6792
Epoch 5/20
264/264 [==============================] - 13s 47ms/step - loss: 0.9369 - accuracy: 0.7992 - val_loss: 0.4152 - val_accuracy: 0.8448
Epoch 6/20
264/264 [==============================] - 12s 46ms/step - loss: 0.8817 - accuracy: 0.8096 - val_loss: 0.5679 - val_accuracy: 0.6773
Epoch 7/20
264/264 [==============================] - 12s 46ms/step - loss: 0.8665 - accuracy: 0.8154 - val_loss: 0.2652 - val_accuracy:

In [16]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

83/83 [==============================] - 1s 9ms/step - loss: 0.2655 - accuracy: 0.8997
